In [1]:
import numpy as np
from src.utils import *
from src.node import *
from tqdm import tqdm

operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]

In [2]:
problem_0 = np.load("data/problem_0.npz")
x = problem_0["x"]
y = problem_0["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

x.shape: (2, 1000)
y.shape: (1000,)


In [11]:
my_population = create_population(100,3,x.shape[0])

In [12]:
simplified = simplify(my_population, x, y)
print("simplified.shape:", len(simplified))

simplified.shape: 98


In [13]:
cost_pop = cost_population(simplified,x,y)
print("Cost population: ", cost_pop.mean())
print("Best cost: ", cost_pop.min())

Cost population:  2703461.7024304005
Best cost:  0.4544972183196075


In [ ]:
NUM_GENERATIONS = 10000
operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]

for _ in tqdm(range(NUM_GENERATIONS)):
    best_ind, worst_ind = tournament_selection(cost_pop,10)
    
    # Xover the best individual and replace worst individuals
    for j in range(0,len(worst_ind)//2,2):
        parent1 = my_population[best_ind[np.random.choice(len(worst_ind))]]
        parent2 = my_population[best_ind[np.random.choice(len(worst_ind))]]
        child1, child2 = crossover(parent1,parent2)
        

        my_population[worst_ind[j]] = child1
        my_population[worst_ind[j+1]] = child2
    

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:06<00:00, 1607.19it/s]


In [11]:
my_population = simplify(my_population, x, y)
cost_pop = cost_population(my_population,x,y)
print("Cost population: ", cost_pop.mean())
print("Best cost: ", cost_pop.min())

Cost population:  67.91755868390331
Best cost:  0.21878415413758193


np.float64(-0.6805286532993463)